In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import matplotlib as mpl
import os
from toolz import concat
from matplotlib.lines import Line2D
from collections import Counter
import math
from tqdm import tqdm
%matplotlib inline
import warnings
warnings.simplefilter('ignore')
import random
import scipy
from aging.plotting import format_plots, PlotConfig, save_factory, figure, legend, format_pizza_plots
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tools.tools import pinv_extended  
from statsmodels.stats.anova import anova_lm
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from tabulate import tabulate

In [2]:
format_plots()
#format_pizza_plots()

In [3]:
def mm_norm_col(column):
    return (column - column.min()) / (column.max() - column.min())

## arrange data for ontogeny - male and female seperately

In [4]:
## arrange data
keep_syllables = np.loadtxt('/n/groups/datta/win/longtogeny/data/ontogeny/version_11/to_keep_syllables_raw.txt', dtype=int)

male_df = pd.read_parquet('/n/groups/datta/win/longtogeny/data/ontogeny/version_11-1/beh_age_df/2024-07-29-ontogeny_males_raw_usage_df_beh_age.parquet').astype(float)
male_df = male_df[keep_syllables]

# avarage per age or session
m_df = male_df.groupby(['age','uuid']).mean()

## arrange data for females
keep_syllables = np.loadtxt('/n/groups/datta/win/longtogeny/data/ontogeny/version_11/to_keep_syllables_raw.txt', dtype=int)

female_df = pd.read_parquet('/n/groups/datta/win/longtogeny/data/ontogeny/version_11-1/beh_age_df/2024-07-29-ontogeny_females_raw_usage_df_beh_age.parquet').astype(float)
female_df = female_df[keep_syllables]


# for female:
f_df= female_df.groupby(['age','uuid']).mean()

In [5]:
def tw_anova(data, dependent_var, factor1, factor2):
    """
    Perform two-way ANOVA.
    
    Parameters:
    data (pd.DataFrame): The input data frame.
    dependent_var (str): The name of the dependent variable column.
    factor1 (str): The name of the first factor column.
    factor2 (str): The name of the second factor column.
    
    Returns:
    pd.DataFrame: The ANOVA results.
    """
    
    # Align the data
    align_data = data.copy()
    align_data['ranked_response'] = align_data[[dependent_var]]
    # Fit an OLS model on the ranked data
    #model = ols(f'ranked_response ~ {factor1} * {factor2}', data=align_data).fit()
    model = ols(f'ranked_response ~ {factor1} * C({factor2})', data=align_data).fit()

    # Perform ANOVA on the fitted model
    anova_results = sm.stats.anova_lm(model, typ=3)
    
    # Convert results to a DataFrame
    anova_df = pd.DataFrame(anova_results)
    
    return anova_df

def ow_anova(data, dependent_var, factor1):
    """
    Perform two-way ANOVA.
    
    Parameters:
    data (pd.DataFrame): The input data frame.
    dependent_var (str): The name of the dependent variable column.
    factor1 (str): The name of the first factor column.
    factor2 (str): The name of the second factor column.
    
    Returns:
    pd.DataFrame: The ANOVA results.
    """
    
    # Align the data
    align_data = data.copy()
    align_data['ranked_response'] = align_data[[dependent_var]]
    # Fit an OLS model on the ranked data
    model = ols(f'ranked_response ~ {factor1}', data=align_data).fit()
    # Perform ANOVA on the fitted model
    anova_results = sm.stats.anova_lm(model, typ=3)
    
    # Convert results to a DataFrame
    anova_df = pd.DataFrame(anova_results)
    
    return anova_df

## arrange data for male and female together

In [6]:
# use only common ages
f_ages = f_df.index.get_level_values('age').unique().to_numpy()
m_ages = m_df.index.get_level_values('age').unique().to_numpy()
cages = list(np.intersect1d(f_ages,m_ages))

m_data = m_df.loc[m_df.index.get_level_values('age').isin(cages)]
m_data['sex'] = 'm'
m_data.set_index('sex',inplace=True,append=True)

f_data = f_df.loc[f_df.index.get_level_values('age').isin(cages)]
f_data['sex'] = 'f'
f_data.set_index('sex',inplace=True,append=True)
data = pd.concat([m_data,f_data])

## change in syllable over age males - one-way anova

In [7]:
#males
data=m_df
temp = data.copy()
temp.reset_index(inplace=True)
results=[]
if 'level_0' in temp.columns:
    temp.drop(columns=['level_0'], inplace=True)

syll = data.columns
n=len(syll)
s=[]
p=[]
h=[]
a = 0.05
from scipy import stats
for i in syll:
    print(i)
    anova_results = ow_anova(temp, i, 'age')
    #results[i] = anova_results
    #print(anova_results)
    print(tabulate(anova_results, headers='keys', tablefmt='psql'))
    cp = anova_results['PR(>F)']['age']*n
    if cp<a:
        s.extend([i])
        p.extend([cp])
        h.extend([h])
print('number of syllables changing over time for males:', str(len(s)))

0
+-----------+-------------+------+----------+---------------+
|           |      sum_sq |   df |        F |        PR(>F) |
|-----------+-------------+------+----------+---------------|
| Intercept | 0.0016897   |    1 | 157.083  |   9.3541e-30  |
| age       | 0.000253767 |    1 |  23.5914 |   1.84079e-06 |
| Residual  | 0.00356049  |  331 | nan      | nan           |
+-----------+-------------+------+----------+---------------+
2
+-----------+-----------+------+----------+---------------+
|           |    sum_sq |   df |        F |        PR(>F) |
|-----------+-----------+------+----------+---------------|
| Intercept | 0.127869  |    1 | 559.37   |   4.16375e-73 |
| age       | 0.0155289 |    1 |  67.9323 |   4.0058e-15  |
| Residual  | 0.0756647 |  331 | nan      | nan           |
+-----------+-----------+------+----------+---------------+
4
+-----------+-------------+------+------------+---------------+
|           |      sum_sq |   df |          F |        PR(>F) |
|-----------

+-----------+------------+------+------------+---------------+
|           |     sum_sq |   df |          F |        PR(>F) |
|-----------+------------+------+------------+---------------|
| Intercept | 0.0278629  |    1 | 170.941    |   8.84467e-32 |
| age       | 5.7184e-05 |    1 |   0.350828 |   0.554049    |
| Residual  | 0.0539522  |  331 | nan        | nan           |
+-----------+------------+------+------------+---------------+
54
+-----------+-------------+------+----------+---------------+
|           |      sum_sq |   df |        F |        PR(>F) |
|-----------+-------------+------+----------+---------------|
| Intercept | 0.000754504 |    1 |  86.9682 |   1.62214e-18 |
| age       | 0.000170313 |    1 |  19.6312 |   1.27843e-05 |
| Residual  | 0.00287163  |  331 | nan      | nan           |
+-----------+-------------+------+----------+---------------+
55
+-----------+-------------+------+------------+--------------+
|           |      sum_sq |   df |          F |       PR

In [8]:
#males
data=m_df
temp = data.copy()
temp.reset_index(inplace=True)
results=[]
if 'level_0' in temp.columns:
    temp.drop(columns=['level_0'], inplace=True)

# Initialize variables
results = []
dependent_vars = data.columns
n_tests = len(dependent_vars)  # Number of ANOVA tests

# Perform ANOVA for each dependent variable
for i, k in enumerate(dependent_vars):
    print(k)
    anova_results = ow_anova(temp, k, 'age')
    anova_results['Dependent Variable'] = k  # Add dependent variable to the results
    results.append(anova_results)
    print(tabulate(anova_results, headers='keys', tablefmt='psql'))

# Combine results into a DataFrame
results_df = pd.concat(results).reset_index()
results_df.rename(columns={'index': 'Parameter'}, inplace=True)

# Move the 'Dependent Variable' column to the first position
cols = ['Dependent Variable'] + [col for col in results_df.columns if col != 'Dependent Variable']
results_df = results_df[cols]

# Apply Bonferroni correction to the p-values
results_df['Bonferroni_corrected_p'] = results_df['PR(>F)'] * n_tests
results_df['Bonferroni_corrected_p'] = results_df['Bonferroni_corrected_p'].apply(lambda p: min(p, 1))  # p-values should not exceed 1

# Save DataFrame to CSV file
results_df.to_csv('/n/groups/datta/win/longtogeny/data/ontogeny/version_11-1/stats/anova_results_syllable_over_time_ontogeny_males_beh_age.csv', index=False)

print("ANOVA results with Bonferroni-corrected p-values have been saved to 'anova_results.csv'")

0
+-----------+-------------+------+----------+---------------+----------------------+
|           |      sum_sq |   df |        F |        PR(>F) |   Dependent Variable |
|-----------+-------------+------+----------+---------------+----------------------|
| Intercept | 0.0016897   |    1 | 157.083  |   9.3541e-30  |                    0 |
| age       | 0.000253767 |    1 |  23.5914 |   1.84079e-06 |                    0 |
| Residual  | 0.00356049  |  331 | nan      | nan           |                    0 |
+-----------+-------------+------+----------+---------------+----------------------+
2
+-----------+-----------+------+----------+---------------+----------------------+
|           |    sum_sq |   df |        F |        PR(>F) |   Dependent Variable |
|-----------+-----------+------+----------+---------------+----------------------|
| Intercept | 0.127869  |    1 | 559.37   |   4.16375e-73 |                    2 |
| age       | 0.0155289 |    1 |  67.9323 |   4.0058e-15  |          

+-----------+-----------+------+---------+---------------+----------------------+
|           |    sum_sq |   df |       F |        PR(>F) |   Dependent Variable |
|-----------+-----------+------+---------+---------------+----------------------|
| Intercept | 0.0130677 |    1 | 156.11  |   1.30432e-29 |                   24 |
| age       | 0.002065  |    1 |  24.669 |   1.09281e-06 |                   24 |
| Residual  | 0.0277075 |  331 | nan     | nan           |                   24 |
+-----------+-----------+------+---------+---------------+----------------------+
25
+-----------+------------+------+----------+---------------+----------------------+
|           |     sum_sq |   df |        F |        PR(>F) |   Dependent Variable |
|-----------+------------+------+----------+---------------+----------------------|
| Intercept | 0.00140889 |    1 |  90.3336 |   4.23975e-19 |                   25 |
| age       | 0.00122479 |    1 |  78.5296 |   4.93527e-17 |                   25 |
| R

+-----------+------------+------+----------+---------------+----------------------+
|           |     sum_sq |   df |        F |        PR(>F) |   Dependent Variable |
|-----------+------------+------+----------+---------------+----------------------|
| Intercept | 0.00311863 |    1 |  12.6117 |   0.000438932 |                   59 |
| age       | 0.0160687  |    1 |  64.9814 |   1.39345e-14 |                   59 |
| Residual  | 0.0818502  |  331 | nan      | nan           |                   59 |
+-----------+------------+------+----------+---------------+----------------------+
60
+-----------+-------------+------+-----------+---------------+----------------------+
|           |      sum_sq |   df |         F |        PR(>F) |   Dependent Variable |
|-----------+-------------+------+-----------+---------------+----------------------|
| Intercept | 0.00264775  |    1 | 337.388   |   1.89734e-52 |                   60 |
| age       | 5.69966e-05 |    1 |   7.26277 |   0.00739921  |   

+-----------+-------------+------+------------+--------------+----------------------+
|           |      sum_sq |   df |          F |       PR(>F) |   Dependent Variable |
|-----------+-------------+------+------------+--------------+----------------------|
| Intercept | 0.0115454   |    1 | 160.41     |   3.0183e-30 |                   79 |
| age       | 1.96417e-05 |    1 |   0.272898 |   0.601743   |                   79 |
| Residual  | 0.0238235   |  331 | nan        | nan          |                   79 |
+-----------+-------------+------+------------+--------------+----------------------+
83
+-----------+------------+------+---------+---------------+----------------------+
|           |     sum_sq |   df |       F |        PR(>F) |   Dependent Variable |
|-----------+------------+------+---------+---------------+----------------------|
| Intercept | 0.00412497 |    1 | 136.183 |   1.37307e-26 |                   83 |
| age       | 0.00110813 |    1 |  36.584 |   3.95046e-09 |    

In [9]:
#females
data=f_df
temp = data.copy()
temp.reset_index(inplace=True)
results=[]
if 'level_0' in temp.columns:
    temp.drop(columns=['level_0'], inplace=True)

syll = data.columns
n=len(syll)
s=[]
p=[]
h=[]
a = 0.05
from scipy import stats
for i in syll:
    print(i)
    anova_results = ow_anova(temp, i, 'age')
    #results[i] = anova_results
    #print(anova_results)
    print(tabulate(anova_results, headers='keys', tablefmt='psql'))
    cp = anova_results['PR(>F)']['age']*n
    if cp<a:
        s.extend([i])
        p.extend([cp])
print('number of syllables changing over time for females:', str(len(s)))

0
+-----------+------------+------+----------+---------------+
|           |     sum_sq |   df |        F |        PR(>F) |
|-----------+------------+------+----------+---------------|
| Intercept | 0.00138464 |    1 |  74.6293 |   2.89733e-16 |
| age       | 0.00141416 |    1 |  76.2205 |   1.51232e-16 |
| Residual  | 0.00584437 |  315 | nan      | nan           |
+-----------+------------+------+----------+---------------+
2
+-----------+------------+------+-----------+---------------+
|           |     sum_sq |   df |         F |        PR(>F) |
|-----------+------------+------+-----------+---------------|
| Intercept | 0.0948507  |    1 | 345.368   |   1.44402e-52 |
| age       | 0.00036313 |    1 |   1.32222 |   0.251067    |
| Residual  | 0.0865105  |  315 | nan       | nan           |
+-----------+------------+------+-----------+---------------+
4
+-----------+------------+------+-----------+---------------+
|           |     sum_sq |   df |         F |        PR(>F) |
|--------

+-----------+------------+------+------------+---------------+
|           |     sum_sq |   df |          F |        PR(>F) |
|-----------+------------+------+------------+---------------|
| Intercept | 0.00397405 |    1 | 162.526    |   2.66529e-30 |
| age       | 4.5938e-06 |    1 |   0.187872 |   0.66499     |
| Residual  | 0.00770231 |  315 | nan        | nan           |
+-----------+------------+------+------------+---------------+
36
+-----------+-------------+------+--------------+---------------+
|           |      sum_sq |   df |            F |        PR(>F) |
|-----------+-------------+------+--------------+---------------|
| Intercept | 0.118125    |    1 | 970.311      |   3.37424e-98 |
| age       | 2.85678e-07 |    1 |   0.00234663 |   0.961395    |
| Residual  | 0.038348    |  315 | nan          | nan           |
+-----------+-------------+------+--------------+---------------+
39
+-----------+-------------+------+----------+---------------+
|           |      sum_sq |  

+-----------+-------------+------+----------+---------------+
|           |      sum_sq |   df |        F |        PR(>F) |
|-----------+-------------+------+----------+---------------|
| Intercept | 0.0020349   |    1 |  99.6397 |   1.44074e-20 |
| age       | 0.000328863 |    1 |  16.1029 |   7.50136e-05 |
| Residual  | 0.00643313  |  315 | nan      | nan           |
+-----------+-------------+------+----------+---------------+
84
+-----------+-------------+------+-----------+--------------+
|           |      sum_sq |   df |         F |       PR(>F) |
|-----------+-------------+------+-----------+--------------|
| Intercept | 0.00515125  |    1 | 246.663   |   1.8656e-41 |
| age       | 0.000145221 |    1 |   6.95379 |   0.00877892 |
| Residual  | 0.00657837  |  315 | nan       | nan          |
+-----------+-------------+------+-----------+--------------+
85
+-----------+-------------+------+----------+---------------+
|           |      sum_sq |   df |        F |        PR(>F) |
|-

In [10]:
#males
data=f_df
temp = data.copy()
temp.reset_index(inplace=True)
results=[]
if 'level_0' in temp.columns:
    temp.drop(columns=['level_0'], inplace=True)

# Initialize variables
results = []
dependent_vars = data.columns
n_tests = len(dependent_vars)  # Number of ANOVA tests

# Perform ANOVA for each dependent variable
for i, k in enumerate(dependent_vars):
    print(k)
    anova_results = ow_anova(temp, k, 'age')
    anova_results['Dependent Variable'] = k  # Add dependent variable to the results
    results.append(anova_results)
    print(tabulate(anova_results, headers='keys', tablefmt='psql'))

# Combine results into a DataFrame
results_df = pd.concat(results).reset_index()
results_df.rename(columns={'index': 'Parameter'}, inplace=True)

# Move the 'Dependent Variable' column to the first position
cols = ['Dependent Variable'] + [col for col in results_df.columns if col != 'Dependent Variable']
results_df = results_df[cols]

# Apply Bonferroni correction to the p-values
results_df['Bonferroni_corrected_p'] = results_df['PR(>F)'] * n_tests
results_df['Bonferroni_corrected_p'] = results_df['Bonferroni_corrected_p'].apply(lambda p: min(p, 1))  # p-values should not exceed 1

# Save DataFrame to CSV file
results_df.to_csv('/n/groups/datta/win/longtogeny/data/ontogeny/version_11-1/stats/anova_results_syllable_over_time_ontogeny_females_beh_age.csv', index=False)

print("ANOVA results with Bonferroni-corrected p-values have been saved to 'anova_results.csv'")

0
+-----------+------------+------+----------+---------------+----------------------+
|           |     sum_sq |   df |        F |        PR(>F) |   Dependent Variable |
|-----------+------------+------+----------+---------------+----------------------|
| Intercept | 0.00138464 |    1 |  74.6293 |   2.89733e-16 |                    0 |
| age       | 0.00141416 |    1 |  76.2205 |   1.51232e-16 |                    0 |
| Residual  | 0.00584437 |  315 | nan      | nan           |                    0 |
+-----------+------------+------+----------+---------------+----------------------+
2
+-----------+------------+------+-----------+---------------+----------------------+
|           |     sum_sq |   df |         F |        PR(>F) |   Dependent Variable |
|-----------+------------+------+-----------+---------------+----------------------|
| Intercept | 0.0948507  |    1 | 345.368   |   1.44402e-52 |                    2 |
| age       | 0.00036313 |    1 |   1.32222 |   0.251067    |       

+-----------+-------------+------+-----------+---------------+----------------------+
|           |      sum_sq |   df |         F |        PR(>F) |   Dependent Variable |
|-----------+-------------+------+-----------+---------------+----------------------|
| Intercept | 0.00397951  |    1 | 177.496   |   2.00502e-32 |                   24 |
| age       | 0.000143559 |    1 |   6.40311 |   0.0118789   |                   24 |
| Residual  | 0.00706238  |  315 | nan       | nan           |                   24 |
+-----------+-------------+------+-----------+---------------+----------------------+
25
+-----------+-------------+------+-----------+---------------+----------------------+
|           |      sum_sq |   df |         F |        PR(>F) |   Dependent Variable |
|-----------+-------------+------+-----------+---------------+----------------------|
| Intercept | 5.20739e-05 |    1 |   3.28163 |   0.0710118   |                   25 |
| age       | 0.0017471   |    1 | 110.1     |   2.

+-----------+------------+------+----------+---------------+----------------------+
|           |     sum_sq |   df |        F |        PR(>F) |   Dependent Variable |
|-----------+------------+------+----------+---------------+----------------------|
| Intercept | 0.0389573  |    1 | 161.316  |   3.98395e-30 |                   53 |
| age       | 0.00262379 |    1 |  10.8647 |   0.00109203  |                   53 |
| Residual  | 0.0760714  |  315 | nan      | nan           |                   53 |
+-----------+------------+------+----------+---------------+----------------------+
54
+-----------+-------------+------+----------+---------------+----------------------+
|           |      sum_sq |   df |        F |        PR(>F) |   Dependent Variable |
|-----------+-------------+------+----------+---------------+----------------------|
| Intercept | 0.000782397 |    1 |  71.5284 |   1.03702e-15 |                   54 |
| age       | 0.00076672  |    1 |  70.0952 |   1.87655e-15 |        

+-----------+-------------+------+-----------+---------------+----------------------+
|           |      sum_sq |   df |         F |        PR(>F) |   Dependent Variable |
|-----------+-------------+------+-----------+---------------+----------------------|
| Intercept | 0.000287417 |    1 |   5.73563 |   0.017208    |                   74 |
| age       | 0.00439347  |    1 |  87.6749  |   1.52142e-18 |                   74 |
| Residual  | 0.0157849   |  315 | nan       | nan           |                   74 |
+-----------+-------------+------+-----------+---------------+----------------------+
76
+-----------+-------------+------+----------+--------------+----------------------+
|           |      sum_sq |   df |        F |       PR(>F) |   Dependent Variable |
|-----------+-------------+------+----------+--------------+----------------------|
| Intercept | 0.00112119  |    1 | 177.091  |   2.284e-32  |                   76 |
| age       | 0.000323778 |    1 |  51.1405 |   6.0423e-12 

In [11]:
## sex-specific change in syllable over age 2-way anova

In [12]:
data = pd.concat([m_data,f_data])
temp = data.copy()
temp.reset_index(inplace=True)
results=[]
if 'level_0' in temp.columns:
    temp.drop(columns=['level_0'], inplace=True)

syll = data.columns
n=len(syll)
ss=[]
ps=[]
ints=[]
intp=[]
results={}
a = 0.05
from scipy import stats
for i in syll:
    print(i)
    anova_results = tw_anova(temp, i, 'age','sex')
    #print(anova_results)
    print(tabulate(anova_results, headers='keys', tablefmt='psql'))
    #results[i] = anova_results
    cp = anova_results['PR(>F)']['C(sex)']*n
    if cp<a:
        ss.extend([i])
        ps.extend([cp])
    cp = anova_results['PR(>F)']['age:C(sex)']*n
    if cp<a:
        ints.extend([i])
        intp.extend([cp])
print('number of syllables different between males and females:', str(len(ss)))
print('number of syllables with sex/age interactions: ', str(len(ints)))

0
+------------+-------------+------+-------------+---------------+
|            |      sum_sq |   df |           F |        PR(>F) |
|------------+-------------+------+-------------+---------------|
| Intercept  | 0.00138464  |    1 |  93.7479    |   9.15467e-21 |
| C(sex)     | 4.15818e-07 |    1 |   0.0281532 |   0.866803    |
| age        | 0.00141416  |    1 |  95.7467    |   3.8052e-21  |
| age:C(sex) | 0.000260691 |    1 |  17.6503    |   3.0376e-05  |
| Residual   | 0.00931975  |  631 | nan         | nan           |
+------------+-------------+------+-------------+---------------+
2
+------------+-------------+------+-----------+---------------+
|            |      sum_sq |   df |         F |        PR(>F) |
|------------+-------------+------+-----------+---------------|
| Intercept  | 0.0948507   |    1 | 375.822   |   4.92357e-66 |
| C(sex)     | 0.000290445 |    1 |   1.15082 |   0.283788    |
| age        | 0.00036313  |    1 |   1.43881 |   0.230782    |
| age:C(sex) | 0.0

+------------+-------------+------+---------------+----------------+
|            |      sum_sq |   df |             F |         PR(>F) |
|------------+-------------+------+---------------+----------------|
| Intercept  | 0.118125    |    1 | 1119.26       |   6.43672e-142 |
| C(sex)     | 0.00289968  |    1 |   27.475      |   2.17335e-07  |
| age        | 2.85678e-07 |    1 |    0.00270686 |   0.958523     |
| age:C(sex) | 0.00121736  |    1 |   11.5347     |   0.000725972  |
| Residual   | 0.0665949   |  631 |  nan          | nan            |
+------------+-------------+------+---------------+----------------+
39
+------------+-------------+------+-----------+---------------+
|            |      sum_sq |   df |         F |        PR(>F) |
|------------+-------------+------+-----------+---------------|
| Intercept  | 0.000312919 |    1 |  12.1008  |   0.00053867  |
| C(sex)     | 0.000242002 |    1 |   9.35843 |   0.00231371  |
| age        | 0.000124954 |    1 |   4.83207 |   0.0282

+------------+-------------+------+-----------+---------------+
|            |      sum_sq |   df |         F |        PR(>F) |
|------------+-------------+------+-----------+---------------|
| Intercept  | 0.000287417 |    1 |   2.31815 |   0.128373    |
| C(sex)     | 0.000559489 |    1 |   4.51252 |   0.0340358   |
| age        | 0.00439347  |    1 |  35.4352  |   4.37611e-09 |
| age:C(sex) | 0.0171773   |    1 | 138.542   |   4.72018e-29 |
| Residual   | 0.0782351   |  631 | nan       | nan           |
+------------+-------------+------+-----------+---------------+
76
+------------+-------------+------+-----------+---------------+
|            |      sum_sq |   df |         F |        PR(>F) |
|------------+-------------+------+-----------+---------------|
| Intercept  | 0.00112119  |    1 | 108.939   |   1.23549e-23 |
| C(sex)     | 0.000295733 |    1 |  28.7345  |   1.16565e-07 |
| age        | 0.000323778 |    1 |  31.4595  |   3.04892e-08 |
| age:C(sex) | 3.64766e-05 |    1 |  

In [13]:
data = pd.concat([m_data,f_data])
temp = data.copy()
temp.reset_index(inplace=True)
results=[]
if 'level_0' in temp.columns:
    temp.drop(columns=['level_0'], inplace=True)

syll = data.columns
n=len(syll)
results={}
a = 0.05

# Initialize variables
results = []
dependent_vars = data.columns
n_tests = len(dependent_vars)  # Number of ANOVA tests

# Perform ANOVA for each dependent variable
for i, k in enumerate(dependent_vars):
    print(k)
    anova_results = tw_anova(temp, k, 'age', 'sex')
    anova_results['Dependent Variable'] = k  # Add dependent variable to the results
    results.append(anova_results)
    print(tabulate(anova_results, headers='keys', tablefmt='psql'))

# Combine results into a DataFrame
results_df = pd.concat(results).reset_index()
results_df.rename(columns={'index': 'Parameter'}, inplace=True)

# Move the 'Dependent Variable' column to the first position
cols = ['Dependent Variable'] + [col for col in results_df.columns if col != 'Dependent Variable']
results_df = results_df[cols]

# Apply Bonferroni correction to the p-values
results_df['Bonferroni_corrected_p'] = results_df['PR(>F)'] * n_tests
results_df['Bonferroni_corrected_p'] = results_df['Bonferroni_corrected_p'].apply(lambda p: min(p, 1))  # p-values should not exceed 1

# Save DataFrame to CSV file
results_df.to_csv('/n/groups/datta/win/longtogeny/data/ontogeny/version_11-1/stats/anova_results_fvm_ontogeny_beh_age.csv', index=False)

print("ANOVA results with Bonferroni-corrected p-values have been saved to 'anova_results.csv'")

0
+------------+-------------+------+-------------+---------------+----------------------+
|            |      sum_sq |   df |           F |        PR(>F) |   Dependent Variable |
|------------+-------------+------+-------------+---------------+----------------------|
| Intercept  | 0.00138464  |    1 |  93.7479    |   9.15467e-21 |                    0 |
| C(sex)     | 4.15818e-07 |    1 |   0.0281532 |   0.866803    |                    0 |
| age        | 0.00141416  |    1 |  95.7467    |   3.8052e-21  |                    0 |
| age:C(sex) | 0.000260691 |    1 |  17.6503    |   3.0376e-05  |                    0 |
| Residual   | 0.00931975  |  631 | nan         | nan           |                    0 |
+------------+-------------+------+-------------+---------------+----------------------+
2
+------------+-------------+------+-----------+---------------+----------------------+
|            |      sum_sq |   df |         F |        PR(>F) |   Dependent Variable |
|------------+-------

+------------+-------------+------+------------+---------------+----------------------+
|            |      sum_sq |   df |          F |        PR(>F) |   Dependent Variable |
|------------+-------------+------+------------+---------------+----------------------|
| Intercept  | 0.0212104   |    1 | 281.511    |   1.62072e-52 |                   14 |
| C(sex)     | 2.89876e-05 |    1 |   0.384733 |   0.535305    |                   14 |
| age        | 1.57552e-05 |    1 |   0.209108 |   0.647625    |                   14 |
| age:C(sex) | 0.00118662  |    1 |  15.7493   |   8.06226e-05 |                   14 |
| Residual   | 0.0475425   |  631 | nan        | nan           |                   14 |
+------------+-------------+------+------------+---------------+----------------------+
16
+------------+-------------+------+------------+---------------+----------------------+
|            |      sum_sq |   df |          F |        PR(>F) |   Dependent Variable |
|------------+-------------+-

+------------+-------------+------+---------------+----------------+----------------------+
|            |      sum_sq |   df |             F |         PR(>F) |   Dependent Variable |
|------------+-------------+------+---------------+----------------+----------------------|
| Intercept  | 0.118125    |    1 | 1119.26       |   6.43672e-142 |                   36 |
| C(sex)     | 0.00289968  |    1 |   27.475      |   2.17335e-07  |                   36 |
| age        | 2.85678e-07 |    1 |    0.00270686 |   0.958523     |                   36 |
| age:C(sex) | 0.00121736  |    1 |   11.5347     |   0.000725972  |                   36 |
| Residual   | 0.0665949   |  631 |  nan          | nan            |                   36 |
+------------+-------------+------+---------------+----------------+----------------------+
39
+------------+-------------+------+-----------+---------------+----------------------+
|            |      sum_sq |   df |         F |        PR(>F) |   Dependent Variab

+------------+-------------+------+--------------+---------------+----------------------+
|            |      sum_sq |   df |            F |        PR(>F) |   Dependent Variable |
|------------+-------------+------+--------------+---------------+----------------------|
| Intercept  | 0.00228165  |    1 | 343.857      |   1.334e-61   |                   60 |
| C(sex)     | 1.12404e-08 |    1 |   0.00169398 |   0.967183    |                   60 |
| age        | 0.000161799 |    1 |  24.3841     |   1.01165e-06 |                   60 |
| age:C(sex) | 2.0499e-05  |    1 |   3.08931    |   0.0792931   |                   60 |
| Residual   | 0.00418697  |  631 | nan          | nan           |                   60 |
+------------+-------------+------+--------------+---------------+----------------------+
61
+------------+-------------+------+-----------+---------------+----------------------+
|            |      sum_sq |   df |         F |        PR(>F) |   Dependent Variable |
|------------

+------------+-------------+------+-----------+---------------+----------------------+
|            |      sum_sq |   df |         F |        PR(>F) |   Dependent Variable |
|------------+-------------+------+-----------+---------------+----------------------|
| Intercept  | 0.000287417 |    1 |   2.31815 |   0.128373    |                   74 |
| C(sex)     | 0.000559489 |    1 |   4.51252 |   0.0340358   |                   74 |
| age        | 0.00439347  |    1 |  35.4352  |   4.37611e-09 |                   74 |
| age:C(sex) | 0.0171773   |    1 | 138.542   |   4.72018e-29 |                   74 |
| Residual   | 0.0782351   |  631 | nan       | nan           |                   74 |
+------------+-------------+------+-----------+---------------+----------------------+
76
+------------+-------------+------+-----------+---------------+----------------------+
|            |      sum_sq |   df |         F |        PR(>F) |   Dependent Variable |
|------------+-------------+------+-----

ANOVA results with Bonferroni-corrected p-values have been saved to 'anova_results.csv'
